In [1]:
import arcpy
import pandas as pd
import psycopg2
from psycopg2 import sql
import numpy as np
import os
import folium

In [2]:
cwd = os.getcwd()

# Make it workspace

arcpy.env.workspace = os.path.join(cwd, '..', '..', 'data', 'QAQC.gdb')

arcpy.env.overwriteOutput = True # Overwrite layers is okay

save_path_csv = os.path.join(cwd, '..', '..', 'data')
save_path_gdb = os.path.join(cwd, '..', '..', 'data', 'QAQC.gdb')
historic_path = os.path.join(cwd, '..', '..', 'data', 'purpleair_historic.csv')
station_path = os.path.join(cwd, '..', '..', 'data', 'purpleair_stations.csv')
boundary_path = os.path.join(cwd, '..', '..', 'data', 'mpls_boundary.geojson')

In [3]:
output_fc = 'mpls_boundary'
output_fc_path = os.path.join(save_path_gdb, output_fc)

# Convert the GeoJSON file to a feature class
arcpy.JSONToFeatures_conversion(boundary_path, output_fc)

<Result 'C:\\Users\\tande\\Documents\\GitHub\\QualityAirQualityCities\\arcpy\\interpolation\\..\\..\\data\\QAQC.gdb\\mpls_boundary'>

In [4]:
df = pd.read_csv(historic_path)

# Convert the 'timestamp' column to a datetime object
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Filter the rows that have a timestamp of 4/1/23
date_filter = df['timestamp'].dt.date == pd.to_datetime('4/1/23').date()
filtered_df = df[date_filter]

filtered_df.to_csv('historic_april1.csv', index=False)
historic_filtered_path = os.path.join(cwd, '..', '..', 'data', 'historic_april1.csv')

In [5]:
# Load the first CSV file into a DataFrame
df1 = pd.read_csv(station_path)

# Load the second CSV file into a DataFrame
df2 = pd.read_csv(historic_filtered_path)

# Merge the two DataFrames together on a common column
merged_df = pd.merge(df1, df2, on='sensor_index')

# Save the merged DataFrame to a new CSV file
merged_df.to_csv('merged_file.csv', index=False)
merged_path = os.path.join(cwd, '..', '..', 'data', 'merged_file.csv')

In [6]:
# Set output point feature class name and path
output_point_fc = "stations_XYTableToPoint"
output_point_path = os.path.join(save_path_gdb, output_point_fc)

# Specify x,y fields
x_field = "longitude"
y_field = "latitude"

# Use XYTableToPoint tool to create point features
station_points = arcpy.management.XYTableToPoint(merged_path, output_point_path, x_field, y_field)

In [7]:
'''
output_table = "joined_table"
arcpy.management.AddJoin(
    in_layer_or_view=station_points,
    in_field="sensor_index",
    join_table=historic_filtered_path,
    join_field="sensor_index",
    join_type="KEEP_ALL",
    index_join_fields="NO_INDEX_JOIN_FIELDS"
)

# Export the joined layer to a new table
historic_apr1 = arcpy.management.CopyRows("stations_XYTableToPoint", output_table)
'''

'\noutput_table = "joined_table"\narcpy.management.AddJoin(\n    in_layer_or_view=station_points,\n    in_field="sensor_index",\n    join_table=historic_filtered_path,\n    join_field="sensor_index",\n    join_type="KEEP_ALL",\n    index_join_fields="NO_INDEX_JOIN_FIELDS"\n)\n\n# Export the joined layer to a new table\nhistoric_apr1 = arcpy.management.CopyRows("stations_XYTableToPoint", output_table)\n'

## PM 2.5 Interpolation

### Kriging PM 2.5

In [217]:
out_kriging_raster = "Kriging_Apr1"
output_raster_path = os.path.join(save_path_gdb, out_kriging_raster)
arcpy.ga.EmpiricalBayesianKriging(
    in_features="stations_XYTableToPoint",
    z_field="pm2_5",
    out_ga_layer="pm2_5_kriging_stats",
    out_raster=out_kriging_raster,
    cell_size=0.000932919999999967,
    transformation_type="NONE",
    max_local_points=100,
    overlap_factor=1,
    number_semivariograms=100,
    search_neighborhood="NBRTYPE=StandardCircular RADIUS=9.87111293434023E-02 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR",
    output_type="PREDICTION",
    quantile_value=0.5,
    threshold_type="EXCEED",
    probability_threshold=None,
    semivariogram_model_type="POWER"
)
output_raster_path = os.path.join(save_path_gdb, out_kriging_raster)

### IDW PM 2.5

In [42]:
out_idw_raster = "IDW_Apr1"
arcpy.ga.IDW(
    in_features="stations_XYTableToPoint",
    z_field="pm2_5",
    out_ga_layer="pm2_5_idw_stats",
    out_raster=out_idw_raster,
    cell_size=10,
    power=2,
    search_neighborhood="NBRTYPE=Standard S_MAJOR=9.87111293434023E-02 S_MINOR=9.87111293434023E-02 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR",
    weight_field=None
)

<Result 'pm2_5_idw_stats'>

### Local Polynomial PM 2.5

In [44]:
out_poly_raster = "Poly_Apr1"
arcpy.ga.LocalPolynomialInterpolation(
    in_features="stations_XYTableToPoint",
    z_field="pm2_5",
    out_ga_layer="pm2_5_poly_stats",
    out_raster=out_poly_raster,
    cell_size=0.000932919999999967,
    power=1,
    search_neighborhood="NBRTYPE=Standard S_MAJOR=9.87111293434023E-02 S_MINOR=9.87111293434023E-02 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR",
    kernel_function="EXPONENTIAL",
    bandwidth=None,
    use_condition_number="NO_USE_CONDITION_NUMBER",
    condition_number=None,
    weight_field=None,
    output_type="PREDICTION"
)

<Result 'pm2_5_poly_stats'>

## Temperature Interpolation

### Kriging Temperature

In [45]:
out_kriging_temp_raster = "Kriging_Temp_Apr1"
arcpy.ga.EmpiricalBayesianKriging(
    in_features="stations_XYTableToPoint",
    z_field="temperature",
    out_ga_layer="temp_kriging_stats",
    out_raster=out_kriging_temp_raster,
    cell_size=0.000932919999999967,
    transformation_type="NONE",
    max_local_points=100,
    overlap_factor=1,
    number_semivariograms=100,
    search_neighborhood="NBRTYPE=StandardCircular RADIUS=9.87111293434023E-02 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR",
    output_type="PREDICTION",
    quantile_value=0.5,
    threshold_type="EXCEED",
    probability_threshold=None,
    semivariogram_model_type="POWER"
)

<Result 'temp_kriging_stats'>

### IDW Temp

In [46]:
out_idw_temp_raster = "IDW_Temp_Apr1"
arcpy.ga.IDW(
    in_features="stations_XYTableToPoint",
    z_field="temperature",
    out_ga_layer="temp_idw_stats",
    out_raster=out_idw_temp_raster,
    cell_size=0.000932919999999967,
    power=2,
    search_neighborhood="NBRTYPE=Standard S_MAJOR=9.87111293434023E-02 S_MINOR=9.87111293434023E-02 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR",
    weight_field=None
)

<Result 'temp_idw_stats'>

### Local Polynomial Temp

In [221]:
out_poly_temp_raster = "Poly_Temp_Apr1"
arcpy.ga.LocalPolynomialInterpolation(
    in_features="stations_XYTableToPoint",
    z_field="temperature",
    out_ga_layer="temp_poly_stats",
    out_raster=out_poly_temp_raster,
    cell_size=0.000932919999999967,
    power=1,
    search_neighborhood="NBRTYPE=Standard S_MAJOR=9.87111293434023E-02 S_MINOR=9.87111293434023E-02 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR",
    kernel_function="EXPONENTIAL",
    bandwidth=None,
    use_condition_number="NO_USE_CONDITION_NUMBER",
    condition_number=None,
    weight_field=None,
    output_type="PREDICTION"
)

<Result 'temp_poly_stats'>

## Cross Validation

In [207]:
rmse_df = pd.DataFrame()

### Kriging PM 2.5

In [ ]:
pm2_5_kriging_fc = "pm2_5_kriging_cross_valid"
pm2_5_kriging_path = os.path.join(save_path_gdb, pm2_5_kriging_fc)
arcpy.ga.CrossValidation(
    in_geostat_layer="pm2_5_kriging_stats",
    out_point_feature_class=pm2_5_kriging_fc
)
pm2_5_kriging_out_fc = "PM2_5_Kriging_Join"
pm2_5_kriging_out_path = os.path.join(save_path_gdb, pm2_5_kriging_out_fc)
arcpy.analysis.SpatialJoin(
    target_features="stations_XYTableToPoint",
    join_features="pm2_5_kriging_cross_valid",
    out_feature_class=pm2_5_kriging_out_fc,
    join_operation="JOIN_ONE_TO_ONE",
    join_type="KEEP_ALL",
    field_mapping='OID_ "OID_" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.OID_,-1,-1;sensor_index "sensor_index" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.sensor_index,-1,-1;last_modified "last_modified" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.last_modified,-1,-1;date_created "date_created" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.date_created,-1,-1;last_seen "last_seen" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.last_seen,-1,-1;name "name" true true false 100 Text 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.name,0,100;location_type "location_type" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.location_type,-1,-1;firmware_version "firmware_version" true true false 30 Text 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.firmware_version,0,30;uptime "uptime" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.uptime,-1,-1;position_rating "position_rating" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.position_rating,-1,-1;latitude "latitude" true true false 8 Double 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.latitude,-1,-1;longitude "longitude" true true false 8 Double 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.longitude,-1,-1;altitude "altitude" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.altitude,-1,-1;channel_state "channel_state" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.channel_state,-1,-1;channel_flags "channel_flags" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.channel_flags,-1,-1;OID1 "OID_" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,purpleair_historic.csv.OID_,-1,-1;sensor_index_1 "sensor_index" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,purpleair_historic.csv.sensor_index,-1,-1;timestamp "timestamp" true true false 8 Date 0 0,First,#,stations_XYTableToPoint,purpleair_historic.csv.timestamp,-1,-1;humidity "humidity" true true false 8 Double 0 0,First,#,stations_XYTableToPoint,purpleair_historic.csv.humidity,-1,-1;temperature "temperature" true true false 8 Double 0 0,First,#,stations_XYTableToPoint,purpleair_historic.csv.temperature,-1,-1;pressure "pressure" true true false 8 Double 0 0,First,#,stations_XYTableToPoint,purpleair_historic.csv.pressure,-1,-1;pm2_5 "pm2_5" true true false 8 Double 0 0,First,#,stations_XYTableToPoint,purpleair_historic.csv.pm2_5,-1,-1;Measured "Measured" true true false 8 Double 0 0,First,#,pm2_5_kriging_cross_valid,Measured,-1,-1;Predicted "Predicted" true true false 8 Double 0 0,First,#,pm2_5_kriging_cross_valid,Predicted,-1,-1;Error "Error" true true false 8 Double 0 0,First,#,pm2_5_kriging_cross_valid,Error,-1,-1;StdError "Standard Error" true true false 8 Double 0 0,First,#,pm2_5_kriging_cross_valid,StdError,-1,-1;Stdd_Error "Standardized Error" true true false 8 Double 0 0,First,#,pm2_5_kriging_cross_valid,Stdd_Error,-1,-1;NormValue "Normal Value" true true false 8 Double 0 0,First,#,pm2_5_kriging_cross_valid,NormValue,-1,-1;CRPS "Continuous Ranked Probability Score" true true false 8 Double 0 0,First,#,pm2_5_kriging_cross_valid,CRPS,-1,-1;QuanVal "Validation Quantile" true true false 8 Double 0 0,First,#,pm2_5_kriging_cross_valid,QuanVal,-1,-1;Interval90 "Inside 90 Percent Interval" true true false 255 Text 0 0,First,#,pm2_5_kriging_cross_valid,Interval90,0,255;Interval95 "Inside 95 Percent Interval" true true false 255 Text 0 0,First,#,pm2_5_kriging_cross_valid,Interval95,0,255;Source_ID "Source ID" true false false 4 Long 0 0,First,#,pm2_5_kriging_cross_valid,Source_ID,-1,-1;Included "Included" true true false 255 Text 0 0,First,#,pm2_5_kriging_cross_valid,Included,0,255',
    match_option="INTERSECT",
    search_radius=None,
    distance_field_name=""
)

<Result 'C:\\Users\\tande\\Documents\\GitHub\\QualityAirQualityCities\\arcpy\\interpolation\\..\\..\\data\\QAQC.gdb\\PM2_5_Kriging_Join'>

In [205]:
# Specify the input feature class and fields to include
input_fc ="PM2_5_Kriging_Join"
fields = ["latitude", "longitude", "OBJECTID", "Error"]

# Convert the feature class to a NumPy array and create a Pandas DataFrame
arr = arcpy.da.TableToNumPyArray(input_fc, fields)
rmse_df = pd.DataFrame(arr)
rmse_df = rmse_df.rename(columns={'Error': 'PM2_5_Kriging_Error'})

,latitude,longitude,OBJECTID,PM2_5_Kriging_Error
0,0.0,0.0,1,1.886304
1,0.0,0.0,2,3.034219
2,0.0,0.0,3,-26.045040
3,0.0,0.0,4,-0.686374
4,0.0,0.0,5,0.576673
...,...,...,...,...
72,0.0,0.0,73,1.762010
73,0.0,0.0,74,3.239165
74,0.0,0.0,75,3.492016
75,0.0,0.0,76,2.384410


### IDW PM 2.5

In [180]:
pm2_5_idw_fc = 'pm2_5_idw_cross_valid'
pm2_5_idw_path = os.path.join(save_path_gdb, pm2_5_idw_fc)
arcpy.ga.CrossValidation(
    in_geostat_layer="pm2_5_idw_stats",
    out_point_feature_class=pm2_5_idw_fc
)
pm2_5_idw_out_fc = "PM2_5_IDW_Join"
pm2_5_idw_out_path = os.path.join(save_path_gdb, pm2_5_idw_out_fc)
arcpy.analysis.SpatialJoin(
    target_features="stations_XYTableToPoint",
    join_features="pm2_5_idw_cross_valid",
    out_feature_class=pm2_5_idw_out_fc,
    join_operation="JOIN_ONE_TO_ONE",
    join_type="KEEP_ALL",
    field_mapping='OID_ "OID_" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.OID_,-1,-1;sensor_index "sensor_index" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.sensor_index,-1,-1;last_modified "last_modified" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.last_modified,-1,-1;date_created "date_created" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.date_created,-1,-1;last_seen "last_seen" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.last_seen,-1,-1;name "name" true true false 100 Text 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.name,0,100;location_type "location_type" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.location_type,-1,-1;firmware_version "firmware_version" true true false 30 Text 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.firmware_version,0,30;uptime "uptime" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.uptime,-1,-1;position_rating "position_rating" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.position_rating,-1,-1;latitude "latitude" true true false 8 Double 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.latitude,-1,-1;longitude "longitude" true true false 8 Double 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.longitude,-1,-1;altitude "altitude" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.altitude,-1,-1;channel_state "channel_state" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.channel_state,-1,-1;channel_flags "channel_flags" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.channel_flags,-1,-1;OID1 "OID_" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,purpleair_historic.csv.OID_,-1,-1;sensor_index_1 "sensor_index" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,purpleair_historic.csv.sensor_index,-1,-1;timestamp "timestamp" true true false 8 Date 0 0,First,#,stations_XYTableToPoint,purpleair_historic.csv.timestamp,-1,-1;humidity "humidity" true true false 8 Double 0 0,First,#,stations_XYTableToPoint,purpleair_historic.csv.humidity,-1,-1;temperature "temperature" true true false 8 Double 0 0,First,#,stations_XYTableToPoint,purpleair_historic.csv.temperature,-1,-1;pressure "pressure" true true false 8 Double 0 0,First,#,stations_XYTableToPoint,purpleair_historic.csv.pressure,-1,-1;pm2_5 "pm2_5" true true false 8 Double 0 0,First,#,stations_XYTableToPoint,purpleair_historic.csv.pm2_5,-1,-1;Measured "Measured" true true false 8 Double 0 0,First,#,pm2_5_kriging_cross_valid,Measured,-1,-1;Predicted "Predicted" true true false 8 Double 0 0,First,#,pm2_5_kriging_cross_valid,Predicted,-1,-1;Error "Error" true true false 8 Double 0 0,First,#,pm2_5_kriging_cross_valid,Error,-1,-1;StdError "Standard Error" true true false 8 Double 0 0,First,#,pm2_5_kriging_cross_valid,StdError,-1,-1;Stdd_Error "Standardized Error" true true false 8 Double 0 0,First,#,pm2_5_kriging_cross_valid,Stdd_Error,-1,-1;NormValue "Normal Value" true true false 8 Double 0 0,First,#,pm2_5_kriging_cross_valid,NormValue,-1,-1;CRPS "Continuous Ranked Probability Score" true true false 8 Double 0 0,First,#,pm2_5_kriging_cross_valid,CRPS,-1,-1;QuanVal "Validation Quantile" true true false 8 Double 0 0,First,#,pm2_5_kriging_cross_valid,QuanVal,-1,-1;Interval90 "Inside 90 Percent Interval" true true false 255 Text 0 0,First,#,pm2_5_kriging_cross_valid,Interval90,0,255;Interval95 "Inside 95 Percent Interval" true true false 255 Text 0 0,First,#,pm2_5_kriging_cross_valid,Interval95,0,255;Source_ID "Source ID" true false false 4 Long 0 0,First,#,pm2_5_kriging_cross_valid,Source_ID,-1,-1;Included "Included" true true false 255 Text 0 0,First,#,pm2_5_kriging_cross_valid,Included,0,255',
    match_option="INTERSECT",
    search_radius=None,
    distance_field_name=""
)

<Result 'C:\\Users\\tande\\Documents\\GitHub\\QualityAirQualityCities\\arcpy\\interpolation\\..\\..\\data\\QAQC.gdb\\PM2_5_IDW_Join'>

In [181]:
# Specify the input feature class and fields to include
input_fc ="PM2_5_IDW_Join"
fields = ["OBJECTID", "Error"]

# Convert the feature class to a NumPy array and create a Pandas DataFrame
arr = arcpy.da.TableToNumPyArray(input_fc, fields)
df = pd.DataFrame(arr)
merged_df = rmse_df.merge(df, on='OBJECTID', how='left')
rmse_df = merged_df.rename(columns={'Error': 'PM2_5_IDW_Error'})

### Local Polynomial PM 2.5

In [182]:
pm2_5_poly_fc = 'pm2_5_poly_cross_valid'
pm2_5_poly_path = os.path.join(save_path_gdb, pm2_5_poly_fc)
arcpy.ga.CrossValidation(
    in_geostat_layer="pm2_5_poly_stats",
    out_point_feature_class=pm2_5_poly_fc
)
pm2_5_poly_out_fc = "PM2_5_Poly_Join"
pm2_5_poly_out_path = os.path.join(save_path_gdb, pm2_5_poly_out_fc)
arcpy.analysis.SpatialJoin(
    target_features="stations_XYTableToPoint",
    join_features="pm2_5_poly_cross_valid",
    out_feature_class=pm2_5_poly_out_fc,
    join_operation="JOIN_ONE_TO_ONE",
    join_type="KEEP_ALL",
    field_mapping='OID_ "OID_" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.OID_,-1,-1;sensor_index "sensor_index" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.sensor_index,-1,-1;last_modified "last_modified" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.last_modified,-1,-1;date_created "date_created" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.date_created,-1,-1;last_seen "last_seen" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.last_seen,-1,-1;name "name" true true false 100 Text 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.name,0,100;location_type "location_type" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.location_type,-1,-1;firmware_version "firmware_version" true true false 30 Text 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.firmware_version,0,30;uptime "uptime" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.uptime,-1,-1;position_rating "position_rating" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.position_rating,-1,-1;latitude "latitude" true true false 8 Double 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.latitude,-1,-1;longitude "longitude" true true false 8 Double 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.longitude,-1,-1;altitude "altitude" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.altitude,-1,-1;channel_state "channel_state" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.channel_state,-1,-1;channel_flags "channel_flags" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.channel_flags,-1,-1;OID1 "OID_" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,purpleair_historic.csv.OID_,-1,-1;sensor_index_1 "sensor_index" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,purpleair_historic.csv.sensor_index,-1,-1;timestamp "timestamp" true true false 8 Date 0 0,First,#,stations_XYTableToPoint,purpleair_historic.csv.timestamp,-1,-1;humidity "humidity" true true false 8 Double 0 0,First,#,stations_XYTableToPoint,purpleair_historic.csv.humidity,-1,-1;temperature "temperature" true true false 8 Double 0 0,First,#,stations_XYTableToPoint,purpleair_historic.csv.temperature,-1,-1;pressure "pressure" true true false 8 Double 0 0,First,#,stations_XYTableToPoint,purpleair_historic.csv.pressure,-1,-1;pm2_5 "pm2_5" true true false 8 Double 0 0,First,#,stations_XYTableToPoint,purpleair_historic.csv.pm2_5,-1,-1;Measured "Measured" true true false 8 Double 0 0,First,#,pm2_5_kriging_cross_valid,Measured,-1,-1;Predicted "Predicted" true true false 8 Double 0 0,First,#,pm2_5_kriging_cross_valid,Predicted,-1,-1;Error "Error" true true false 8 Double 0 0,First,#,pm2_5_kriging_cross_valid,Error,-1,-1;StdError "Standard Error" true true false 8 Double 0 0,First,#,pm2_5_kriging_cross_valid,StdError,-1,-1;Stdd_Error "Standardized Error" true true false 8 Double 0 0,First,#,pm2_5_kriging_cross_valid,Stdd_Error,-1,-1;NormValue "Normal Value" true true false 8 Double 0 0,First,#,pm2_5_kriging_cross_valid,NormValue,-1,-1;CRPS "Continuous Ranked Probability Score" true true false 8 Double 0 0,First,#,pm2_5_kriging_cross_valid,CRPS,-1,-1;QuanVal "Validation Quantile" true true false 8 Double 0 0,First,#,pm2_5_kriging_cross_valid,QuanVal,-1,-1;Interval90 "Inside 90 Percent Interval" true true false 255 Text 0 0,First,#,pm2_5_kriging_cross_valid,Interval90,0,255;Interval95 "Inside 95 Percent Interval" true true false 255 Text 0 0,First,#,pm2_5_kriging_cross_valid,Interval95,0,255;Source_ID "Source ID" true false false 4 Long 0 0,First,#,pm2_5_kriging_cross_valid,Source_ID,-1,-1;Included "Included" true true false 255 Text 0 0,First,#,pm2_5_kriging_cross_valid,Included,0,255',
    match_option="INTERSECT",
    search_radius=None,
    distance_field_name=""
)

<Result 'C:\\Users\\tande\\Documents\\GitHub\\QualityAirQualityCities\\arcpy\\interpolation\\..\\..\\data\\QAQC.gdb\\PM2_5_Poly_Join'>

In [183]:
# Specify the input feature class and fields to include
input_fc ="PM2_5_Poly_Join"
fields = ["OBJECTID", "Error"]

# Convert the feature class to a NumPy array and create a Pandas DataFrame
arr = arcpy.da.TableToNumPyArray(input_fc, fields)
df = pd.DataFrame(arr)
merged_df = rmse_df.merge(df, on='OBJECTID', how='left')
rmse_df = merged_df.rename(columns={'Error': 'PM2_5_Poly_Error'})

### Kriging Temperature

In [184]:
temp_kriging_fc = 'temp_kriging_cross_valid'
temp_kriging_path = os.path.join(save_path_gdb, temp_kriging_fc)
arcpy.ga.CrossValidation(
    in_geostat_layer="temp_kriging_stats",
    out_point_feature_class=temp_kriging_fc
)
temp_kriging_out_fc = "Temp_Kriging_Join"
temp_kriging_out_path = os.path.join(save_path_gdb, temp_kriging_out_fc)
arcpy.analysis.SpatialJoin(
    target_features="stations_XYTableToPoint",
    join_features="temp_kriging_cross_valid",
    out_feature_class=temp_kriging_out_fc,
    join_operation="JOIN_ONE_TO_ONE",
    join_type="KEEP_ALL",
    field_mapping='OID_ "OID_" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.OID_,-1,-1;sensor_index "sensor_index" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.sensor_index,-1,-1;last_modified "last_modified" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.last_modified,-1,-1;date_created "date_created" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.date_created,-1,-1;last_seen "last_seen" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.last_seen,-1,-1;name "name" true true false 100 Text 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.name,0,100;location_type "location_type" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.location_type,-1,-1;firmware_version "firmware_version" true true false 30 Text 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.firmware_version,0,30;uptime "uptime" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.uptime,-1,-1;position_rating "position_rating" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.position_rating,-1,-1;latitude "latitude" true true false 8 Double 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.latitude,-1,-1;longitude "longitude" true true false 8 Double 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.longitude,-1,-1;altitude "altitude" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.altitude,-1,-1;channel_state "channel_state" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.channel_state,-1,-1;channel_flags "channel_flags" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.channel_flags,-1,-1;OID1 "OID_" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,purpleair_historic.csv.OID_,-1,-1;sensor_index_1 "sensor_index" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,purpleair_historic.csv.sensor_index,-1,-1;timestamp "timestamp" true true false 8 Date 0 0,First,#,stations_XYTableToPoint,purpleair_historic.csv.timestamp,-1,-1;humidity "humidity" true true false 8 Double 0 0,First,#,stations_XYTableToPoint,purpleair_historic.csv.humidity,-1,-1;temperature "temperature" true true false 8 Double 0 0,First,#,stations_XYTableToPoint,purpleair_historic.csv.temperature,-1,-1;pressure "pressure" true true false 8 Double 0 0,First,#,stations_XYTableToPoint,purpleair_historic.csv.pressure,-1,-1;pm2_5 "pm2_5" true true false 8 Double 0 0,First,#,stations_XYTableToPoint,purpleair_historic.csv.pm2_5,-1,-1;Measured "Measured" true true false 8 Double 0 0,First,#,pm2_5_kriging_cross_valid,Measured,-1,-1;Predicted "Predicted" true true false 8 Double 0 0,First,#,pm2_5_kriging_cross_valid,Predicted,-1,-1;Error "Error" true true false 8 Double 0 0,First,#,pm2_5_kriging_cross_valid,Error,-1,-1;StdError "Standard Error" true true false 8 Double 0 0,First,#,pm2_5_kriging_cross_valid,StdError,-1,-1;Stdd_Error "Standardized Error" true true false 8 Double 0 0,First,#,pm2_5_kriging_cross_valid,Stdd_Error,-1,-1;NormValue "Normal Value" true true false 8 Double 0 0,First,#,pm2_5_kriging_cross_valid,NormValue,-1,-1;CRPS "Continuous Ranked Probability Score" true true false 8 Double 0 0,First,#,pm2_5_kriging_cross_valid,CRPS,-1,-1;QuanVal "Validation Quantile" true true false 8 Double 0 0,First,#,pm2_5_kriging_cross_valid,QuanVal,-1,-1;Interval90 "Inside 90 Percent Interval" true true false 255 Text 0 0,First,#,pm2_5_kriging_cross_valid,Interval90,0,255;Interval95 "Inside 95 Percent Interval" true true false 255 Text 0 0,First,#,pm2_5_kriging_cross_valid,Interval95,0,255;Source_ID "Source ID" true false false 4 Long 0 0,First,#,pm2_5_kriging_cross_valid,Source_ID,-1,-1;Included "Included" true true false 255 Text 0 0,First,#,pm2_5_kriging_cross_valid,Included,0,255',
    match_option="INTERSECT",
    search_radius=None,
    distance_field_name=""
)

<Result 'C:\\Users\\tande\\Documents\\GitHub\\QualityAirQualityCities\\arcpy\\interpolation\\..\\..\\data\\QAQC.gdb\\Temp_Kriging_Join'>

In [185]:
# Specify the input feature class and fields to include
input_fc ="Temp_Kriging_Join"
fields = ["OBJECTID", "Error"]

# Convert the feature class to a NumPy array and create a Pandas DataFrame
arr = arcpy.da.TableToNumPyArray(input_fc, fields)
df = pd.DataFrame(arr)
merged_df = rmse_df.merge(df, on='OBJECTID', how='left')
rmse_df = merged_df.rename(columns={'Error': 'Temp_Kriging_Error'})

### IDW Temprature

In [186]:
temp_idw_fc = 'temp_idw_cross_valid'
temp_idw_path = os.path.join(save_path_gdb, temp_idw_fc)
arcpy.ga.CrossValidation(
    in_geostat_layer="temp_idw_stats",
    out_point_feature_class=temp_idw_fc
)
temp_idw_out_fc = "Temp_IDW_Join"
temp_idw_out_path = os.path.join(save_path_gdb, temp_idw_out_fc)
arcpy.analysis.SpatialJoin(
    target_features="stations_XYTableToPoint",
    join_features="temp_idw_cross_valid",
    out_feature_class=temp_idw_out_fc,
    join_operation="JOIN_ONE_TO_ONE",
    join_type="KEEP_ALL",
    field_mapping='OID_ "OID_" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.OID_,-1,-1;sensor_index "sensor_index" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.sensor_index,-1,-1;last_modified "last_modified" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.last_modified,-1,-1;date_created "date_created" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.date_created,-1,-1;last_seen "last_seen" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.last_seen,-1,-1;name "name" true true false 100 Text 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.name,0,100;location_type "location_type" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.location_type,-1,-1;firmware_version "firmware_version" true true false 30 Text 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.firmware_version,0,30;uptime "uptime" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.uptime,-1,-1;position_rating "position_rating" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.position_rating,-1,-1;latitude "latitude" true true false 8 Double 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.latitude,-1,-1;longitude "longitude" true true false 8 Double 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.longitude,-1,-1;altitude "altitude" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.altitude,-1,-1;channel_state "channel_state" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.channel_state,-1,-1;channel_flags "channel_flags" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.channel_flags,-1,-1;OID1 "OID_" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,purpleair_historic.csv.OID_,-1,-1;sensor_index_1 "sensor_index" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,purpleair_historic.csv.sensor_index,-1,-1;timestamp "timestamp" true true false 8 Date 0 0,First,#,stations_XYTableToPoint,purpleair_historic.csv.timestamp,-1,-1;humidity "humidity" true true false 8 Double 0 0,First,#,stations_XYTableToPoint,purpleair_historic.csv.humidity,-1,-1;temperature "temperature" true true false 8 Double 0 0,First,#,stations_XYTableToPoint,purpleair_historic.csv.temperature,-1,-1;pressure "pressure" true true false 8 Double 0 0,First,#,stations_XYTableToPoint,purpleair_historic.csv.pressure,-1,-1;pm2_5 "pm2_5" true true false 8 Double 0 0,First,#,stations_XYTableToPoint,purpleair_historic.csv.pm2_5,-1,-1;Measured "Measured" true true false 8 Double 0 0,First,#,pm2_5_kriging_cross_valid,Measured,-1,-1;Predicted "Predicted" true true false 8 Double 0 0,First,#,pm2_5_kriging_cross_valid,Predicted,-1,-1;Error "Error" true true false 8 Double 0 0,First,#,pm2_5_kriging_cross_valid,Error,-1,-1;StdError "Standard Error" true true false 8 Double 0 0,First,#,pm2_5_kriging_cross_valid,StdError,-1,-1;Stdd_Error "Standardized Error" true true false 8 Double 0 0,First,#,pm2_5_kriging_cross_valid,Stdd_Error,-1,-1;NormValue "Normal Value" true true false 8 Double 0 0,First,#,pm2_5_kriging_cross_valid,NormValue,-1,-1;CRPS "Continuous Ranked Probability Score" true true false 8 Double 0 0,First,#,pm2_5_kriging_cross_valid,CRPS,-1,-1;QuanVal "Validation Quantile" true true false 8 Double 0 0,First,#,pm2_5_kriging_cross_valid,QuanVal,-1,-1;Interval90 "Inside 90 Percent Interval" true true false 255 Text 0 0,First,#,pm2_5_kriging_cross_valid,Interval90,0,255;Interval95 "Inside 95 Percent Interval" true true false 255 Text 0 0,First,#,pm2_5_kriging_cross_valid,Interval95,0,255;Source_ID "Source ID" true false false 4 Long 0 0,First,#,pm2_5_kriging_cross_valid,Source_ID,-1,-1;Included "Included" true true false 255 Text 0 0,First,#,pm2_5_kriging_cross_valid,Included,0,255',
    match_option="INTERSECT",
    search_radius=None,
    distance_field_name=""
)

<Result 'C:\\Users\\tande\\Documents\\GitHub\\QualityAirQualityCities\\arcpy\\interpolation\\..\\..\\data\\QAQC.gdb\\Temp_IDW_Join'>

In [187]:
# Specify the input feature class and fields to include
input_fc ="Temp_IDW_Join"
fields = ["OBJECTID", "Error"]

# Convert the feature class to a NumPy array and create a Pandas DataFrame
arr = arcpy.da.TableToNumPyArray(input_fc, fields)
df = pd.DataFrame(arr)
merged_df = rmse_df.merge(df, on='OBJECTID', how='left')
rmse_df = merged_df.rename(columns={'Error': 'Temp_IDW_Error'})

### Local Polynomial Temperature

In [188]:
temp_poly_fc = 'temp_poly_cross_valid'
temp_poly_path = os.path.join(save_path_gdb, temp_poly_fc)
arcpy.ga.CrossValidation(
    in_geostat_layer="temp_poly_stats",
    out_point_feature_class=temp_poly_fc
)
temp_poly_out_fc = "Temp_Poly_Join"
temp_poly_out_path = os.path.join(save_path_gdb, temp_poly_out_fc)
arcpy.analysis.SpatialJoin(
    target_features="stations_XYTableToPoint",
    join_features="temp_poly_cross_valid",
    out_feature_class=temp_poly_out_fc,
    join_operation="JOIN_ONE_TO_ONE",
    join_type="KEEP_ALL",
    field_mapping='OID_ "OID_" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.OID_,-1,-1;sensor_index "sensor_index" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.sensor_index,-1,-1;last_modified "last_modified" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.last_modified,-1,-1;date_created "date_created" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.date_created,-1,-1;last_seen "last_seen" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.last_seen,-1,-1;name "name" true true false 100 Text 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.name,0,100;location_type "location_type" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.location_type,-1,-1;firmware_version "firmware_version" true true false 30 Text 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.firmware_version,0,30;uptime "uptime" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.uptime,-1,-1;position_rating "position_rating" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.position_rating,-1,-1;latitude "latitude" true true false 8 Double 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.latitude,-1,-1;longitude "longitude" true true false 8 Double 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.longitude,-1,-1;altitude "altitude" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.altitude,-1,-1;channel_state "channel_state" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.channel_state,-1,-1;channel_flags "channel_flags" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.channel_flags,-1,-1;OID1 "OID_" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,purpleair_historic.csv.OID_,-1,-1;sensor_index_1 "sensor_index" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,purpleair_historic.csv.sensor_index,-1,-1;timestamp "timestamp" true true false 8 Date 0 0,First,#,stations_XYTableToPoint,purpleair_historic.csv.timestamp,-1,-1;humidity "humidity" true true false 8 Double 0 0,First,#,stations_XYTableToPoint,purpleair_historic.csv.humidity,-1,-1;temperature "temperature" true true false 8 Double 0 0,First,#,stations_XYTableToPoint,purpleair_historic.csv.temperature,-1,-1;pressure "pressure" true true false 8 Double 0 0,First,#,stations_XYTableToPoint,purpleair_historic.csv.pressure,-1,-1;pm2_5 "pm2_5" true true false 8 Double 0 0,First,#,stations_XYTableToPoint,purpleair_historic.csv.pm2_5,-1,-1;Measured "Measured" true true false 8 Double 0 0,First,#,pm2_5_kriging_cross_valid,Measured,-1,-1;Predicted "Predicted" true true false 8 Double 0 0,First,#,pm2_5_kriging_cross_valid,Predicted,-1,-1;Error "Error" true true false 8 Double 0 0,First,#,pm2_5_kriging_cross_valid,Error,-1,-1;StdError "Standard Error" true true false 8 Double 0 0,First,#,pm2_5_kriging_cross_valid,StdError,-1,-1;Stdd_Error "Standardized Error" true true false 8 Double 0 0,First,#,pm2_5_kriging_cross_valid,Stdd_Error,-1,-1;NormValue "Normal Value" true true false 8 Double 0 0,First,#,pm2_5_kriging_cross_valid,NormValue,-1,-1;CRPS "Continuous Ranked Probability Score" true true false 8 Double 0 0,First,#,pm2_5_kriging_cross_valid,CRPS,-1,-1;QuanVal "Validation Quantile" true true false 8 Double 0 0,First,#,pm2_5_kriging_cross_valid,QuanVal,-1,-1;Interval90 "Inside 90 Percent Interval" true true false 255 Text 0 0,First,#,pm2_5_kriging_cross_valid,Interval90,0,255;Interval95 "Inside 95 Percent Interval" true true false 255 Text 0 0,First,#,pm2_5_kriging_cross_valid,Interval95,0,255;Source_ID "Source ID" true false false 4 Long 0 0,First,#,pm2_5_kriging_cross_valid,Source_ID,-1,-1;Included "Included" true true false 255 Text 0 0,First,#,pm2_5_kriging_cross_valid,Included,0,255',
    match_option="INTERSECT",
    search_radius=None,
    distance_field_name=""
)

<Result 'C:\\Users\\tande\\Documents\\GitHub\\QualityAirQualityCities\\arcpy\\interpolation\\..\\..\\data\\QAQC.gdb\\Temp_Poly_Join'>

In [189]:
# Specify the input feature class and fields to include
input_fc ="Temp_Poly_Join"
fields = ["OBJECTID", "Error"]

# Convert the feature class to a NumPy array and create a Pandas DataFrame
arr = arcpy.da.TableToNumPyArray(input_fc, fields)
df = pd.DataFrame(arr)
merged_df = rmse_df.merge(df, on='OBJECTID', how='left')
rmse_df = merged_df.rename(columns={'Error': 'Temp_Poly_Error'})

## Calculate RMSE

In [191]:
rmse_df['SQ_PM2_5_Kriging_Error'] = rmse_df['PM2_5_Kriging_Error'] ** 2
pm2_5_kriging_rmse = numpy.sqrt(rmse_df['SQ_PM2_5_Kriging_Error'].mean())

In [192]:
rmse_df['SQ_PM2_5_IDW_Error'] = rmse_df['PM2_5_IDW_Error'] ** 2
pm2_5_idw_rmse = numpy.sqrt((rmse_df['SQ_PM2_5_IDW_Error'].values).mean())

In [193]:
rmse_df['SQ_PM2_5_Poly_Error'] = rmse_df['PM2_5_Poly_Error'] ** 2
pm2_5_poly_rmse = numpy.sqrt(rmse_df['SQ_PM2_5_Poly_Error'].mean())

In [196]:
pm2_5_compare = pd.DataFrame({'method':['kriging', 'idw', 'local polynomial'],
                            'RMSE':[pm2_5_kriging_rmse, pm2_5_idw_rmse, pm2_5_poly_rmse]})
# Get index of row with lowest value of rmse
min_index = pm2_5_compare['RMSE'].idxmin()

# Get method with lowest rmse
min_id = pm2_5_compare.loc[min_index, 'method']
print("Method with the lowest PM2.5 RMSE value is", min_id)

Method with the lowest RMSE value is kriging


In [198]:
rmse_df['SQ_Temp_Kriging_Error'] = rmse_df['Temp_Kriging_Error'] ** 2
temp_kriging_rmse = numpy.sqrt(rmse_df['SQ_Temp_Kriging_Error'].mean())

In [199]:
rmse_df['SQ_Temp_IDW_Error'] = rmse_df['Temp_IDW_Error'] ** 2
temp_idw_rmse = numpy.sqrt((rmse_df['SQ_Temp_IDW_Error'].values).mean())

In [200]:
rmse_df['SQ_Temp_Poly_Error'] = rmse_df['Temp_Poly_Error'] ** 2
temp_poly_rmse = numpy.sqrt(rmse_df['SQ_Temp_Poly_Error'].mean())

In [201]:
temp_compare = pd.DataFrame({'method':['kriging', 'idw', 'local polynomial'],
                            'RMSE':[temp_kriging_rmse, temp_idw_rmse, temp_poly_rmse]})
# Get index of row with lowest value of rmse
min_index = temp_compare['RMSE'].idxmin()

# Get method with lowest rmse
min_id = temp_compare.loc[min_index, 'method']
print("Method with the lowest Temperature RMSE value is", min_id)

Method with the lowest RMSE value is kriging


## Raster to Point Each Interpolation

### Kriging PM 2.5

In [218]:
out_point_kriging_pm = "Kriging_PM2_5_Raster_to_Point"
output_raster_path = os.path.join(save_path_gdb, out_point_kriging_pm)
arcpy.conversion.RasterToPoint(
    in_raster="Kriging_Apr1",
    out_point_features=out_point_kriging_pm,
    raster_field="Value"
)

<Result 'C:\\Users\\tande\\Documents\\GitHub\\QualityAirQualityCities\\arcpy\\interpolation\\..\\..\\data\\QAQC.gdb\\Kriging_PM2_5_Raster_to_Point'>

### IDW PM 2.5

In [219]:
out_point_idw_pm = "IDW_PM2_5_Raster_to_Point"
output_raster_path = os.path.join(save_path_gdb, out_point_idw_pm)
arcpy.conversion.RasterToPoint(
    in_raster="IDW_Apr1",
    out_point_features=out_point_idw_pm,
    raster_field="Value"
)

<Result 'C:\\Users\\tande\\Documents\\GitHub\\QualityAirQualityCities\\arcpy\\interpolation\\..\\..\\data\\QAQC.gdb\\IDW_PM2_5_Raster_to_Point'>

### Local Polynomial PM 2.5

In [220]:
out_point_poly_pm = "Poly_PM2_5_Raster_to_Point"
output_raster_path = os.path.join(save_path_gdb, out_point_poly_pm)
arcpy.conversion.RasterToPoint(
    in_raster="Poly_Apr1",
    out_point_features=out_point_poly_pm,
    raster_field="Value"
)

<Result 'C:\\Users\\tande\\Documents\\GitHub\\QualityAirQualityCities\\arcpy\\interpolation\\..\\..\\data\\QAQC.gdb\\Poly_PM2_5_Raster_to_Point'>

### Kriging Temp

In [222]:
out_point_kriging_temp = "Kriging_Temp_Raster_to_Point"
output_raster_path = os.path.join(save_path_gdb, out_point_kriging_temp)
arcpy.conversion.RasterToPoint(
    in_raster="Kriging_Temp_Apr1",
    out_point_features=out_point_kriging_temp,
    raster_field="Value"
)

<Result 'C:\\Users\\tande\\Documents\\GitHub\\QualityAirQualityCities\\arcpy\\interpolation\\..\\..\\data\\QAQC.gdb\\Kriging_Temp_Raster_to_Point'>

### IDW Temp

In [225]:
out_point_idw_temp = "IDW_Temp_Raster_to_Point"
output_raster_path = os.path.join(save_path_gdb, out_point_idw_temp)
arcpy.conversion.RasterToPoint(
    in_raster="IDW_Temp_Apr1",
    out_point_features=out_point_idw_temp,
    raster_field="Value"
)

<Result 'C:\\Users\\tande\\Documents\\GitHub\\QualityAirQualityCities\\arcpy\\interpolation\\..\\..\\data\\QAQC.gdb\\IDW_Temp_Raster_to_Point'>

### Local Polynomial Temp

In [227]:
out_point_poly_temp = "Poly_Temp_Raster_to_Point"
output_raster_path = os.path.join(save_path_gdb, out_point_poly_temp)
arcpy.conversion.RasterToPoint(
    in_raster="Poly_Temp_Apr1",
    out_point_features=out_point_poly_temp,
    raster_field="Value"
)

<Result 'C:\\Users\\tande\\Documents\\GitHub\\QualityAirQualityCities\\arcpy\\interpolation\\..\\..\\data\\QAQC.gdb\\Poly_Temp_Raster_to_Point'>

## Clipping Each Interpolation

### Kriging PM 2.5 Clip

In [229]:
out_clip_kriging_pm = "Kriging_PM2_5"
output_clip_path = os.path.join(save_path_gdb, out_clip_kriging_pm)
arcpy.analysis.Clip(
    in_features="Kriging_PM2_5_Raster_to_Point",
    clip_features='mpls_boundary',
    out_feature_class=out_clip_kriging_pm,
    cluster_tolerance=None
)

<Result 'C:\\Users\\tande\\Documents\\GitHub\\QualityAirQualityCities\\arcpy\\interpolation\\..\\..\\data\\QAQC.gdb\\Kriging_PM2_5'>

### IDW PM 2.5 Clip

In [230]:
out_clip_idw_pm = "IDW_PM2_5"
output_clip_path = os.path.join(save_path_gdb, out_clip_idw_pm)
arcpy.analysis.Clip(
    in_features="IDW_PM2_5_Raster_to_Point",
    clip_features='mpls_boundary',
    out_feature_class=out_clip_idw_pm,
    cluster_tolerance=None
)

<Result 'C:\\Users\\tande\\Documents\\GitHub\\QualityAirQualityCities\\arcpy\\interpolation\\..\\..\\data\\QAQC.gdb\\IDW_PM2_5'>

### Local Polynomial PM 2.5 Clip

In [231]:
out_clip_poly_pm = "Poly_PM2_5"
output_clip_path = os.path.join(save_path_gdb, out_clip_poly_pm)
arcpy.analysis.Clip(
    in_features="Poly_PM2_5_Raster_to_Point",
    clip_features='mpls_boundary',
    out_feature_class=out_clip_poly_pm,
    cluster_tolerance=None
)

<Result 'C:\\Users\\tande\\Documents\\GitHub\\QualityAirQualityCities\\arcpy\\interpolation\\..\\..\\data\\QAQC.gdb\\Poly_PM2_5'>

### Kriging Temperature Clip

In [232]:
out_clip_kriging_temp = "Kriging_Temp"
output_clip_path = os.path.join(save_path_gdb, out_clip_kriging_temp)
arcpy.analysis.Clip(
    in_features="Kriging_Temp_Raster_to_Point",
    clip_features='mpls_boundary',
    out_feature_class=out_clip_kriging_temp,
    cluster_tolerance=None
)

<Result 'C:\\Users\\tande\\Documents\\GitHub\\QualityAirQualityCities\\arcpy\\interpolation\\..\\..\\data\\QAQC.gdb\\Kriging_Temp'>

### IDW Temperature Clip

In [233]:
out_clip_idw_temp = "IDW_Temp"
output_clip_path = os.path.join(save_path_gdb, out_clip_idw_temp)
arcpy.analysis.Clip(
    in_features="IDW_Temp_Raster_to_Point",
    clip_features='mpls_boundary',
    out_feature_class=out_clip_idw_temp,
    cluster_tolerance=None
)

<Result 'C:\\Users\\tande\\Documents\\GitHub\\QualityAirQualityCities\\arcpy\\interpolation\\..\\..\\data\\QAQC.gdb\\IDW_Temp'>

### Local Polynomial Clip

In [234]:
out_clip_poly_temp = "Poly_Temp"
output_clip_path = os.path.join(save_path_gdb, out_clip_poly_temp)
arcpy.analysis.Clip(
    in_features="Poly_Temp_Raster_to_Point",
    clip_features='mpls_boundary',
    out_feature_class=out_clip_poly_temp,
    cluster_tolerance=None
)

<Result 'C:\\Users\\tande\\Documents\\GitHub\\QualityAirQualityCities\\arcpy\\interpolation\\..\\..\\data\\QAQC.gdb\\Poly_Temp'>

## Save to Local and Remove Databases

In [ ]:
### Local (Values)

table_name = "Kriging_PM2_5"

if table_name in arcpy.ListTables(): # Delete table if it exists
    arcpy.management.Delete(table_name)

cols_for_gdb = ['OBJECTID', 'SHAPE', 'pointid', 'grid_code']

vals = df_historic[cols_for_gdb].values

dtypes = [('sensor_index', '<U6'), ('timestamp', 'M8[us]'),
          ('humidity', '<f8'), ('temperature', '<f8'),
          ('pressure', '<f8'), ('pm2_5', '<f8')]

array_for_gdb = numpy.rec.fromrecords(vals, names = cols_for_gdb, dtype=dtypes)

arcpy.da.NumPyArrayToTable(array_for_gdb, os.path.join(arcpy.env.workspace,table_name))